<a href="https://colab.research.google.com/github/mukkatharun/ApproxNearestNeighborAlgos/blob/main/Approx_nearest_neighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use state of art libraries for Approximate nearest neighbor search for your favorite data set

In [32]:
!pip install datasketch
!pip install lightfm
!pip install faiss-gpu
!pip install nmslib
!pip install annoy

     |████████████████████████████████| 646 kB 5.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391671 sha256=660a31f5a9596a57ee28e524417ac3b6b55a961827bd4a180d3a7cd3cdab526c
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [33]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest
import csv
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import pickle
import faiss
import nmslib
import annoy

Locality sensitive hashing(LSH)


In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\n]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [ ]:
permutations = 128
num_recommendations = 1
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [ ]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [ ]:
db = pd.read_csv('papers.csv',error_bad_lines=False, engine="python")
db['text'] = db['title'] + ' ' + db['abstract']
forest = get_forest(db, permutations)

Skipping line 4574: unexpected end of data


It took 9.268283605575562 seconds to build forest.


In [ ]:
db.shape

(4572, 8)

In [ ]:
db.head(5)

,id,year,title,event_type,pdf_name,abstract,paper_text,text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,Self-Organization of Associative Database and ...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,A Mean Field Theory of Layer IV of Visual Cort...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,Storing Covariance by the Associative Long-Ter...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...,Bayesian Query Construction for Neural Network...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a...","Neural Network Ensembles, Cross Validation, an..."


In [38]:
num_recommendations = 5
title = 'Bayesian Query Construction for Neural Network'
result = predict(title, db, permutations, 5, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.004976511001586914 seconds to query forest.

 Top Recommendation(s) is(are) 
 3       Bayesian Query Construction for Neural Network...
652     Global Optimisation of Neural Network Models v...
308     Dynamically Adaptable CMOS Winner-Take-All Neu...
859         Neural Network Based Model Predictive Control
2047                         Neural Network Visualization
Name: title, dtype: object


Product Quantization

In [10]:
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
movielens = fetch_movielens()
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [11]:
train = movielens['train']
test = movielens['test']

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.59, test 0.10.
AUC: train 0.89, test 0.86.


In [12]:
train = movielens['train']
test = movielens['test']
item_vectors = movielens['item_features'] * model.item_embeddings

In [14]:
with open('movielens.pickle', 'wb') as f:
    pickle.dump({"name": movielens['item_feature_labels'], "vector": item_vectors}, f)

In [15]:
def load_data():
    with open('movielens.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-2.8606594e-01,  2.3608355e-01,  3.6903396e-01, ...,
          6.0274941e-01, -2.5291806e-01,  5.9300417e-01],
        [ 4.4549662e-01,  6.2510759e-01,  2.2007335e-02, ...,
          3.4654805e-01,  3.3281863e-01, -3.0071938e-01],
        [ 1.5155029e-01,  4.0324977e-01,  4.5176700e-01, ...,
          4.0110886e-01,  3.5056341e-01,  5.8581591e-01],
        ...,
        [-4.7731180e-02, -1.7034501e-01,  5.1332716e-02, ...,
         -1.5277454e-01, -3.1650472e-02,  4.8078176e-02],
        [-5.2377254e-02,  1.3624683e-01,  7.1398795e-02, ...,
          8.5100338e-02,  7.0923008e-02, -5.3595174e-02],
        [-1.9358748e-04,  1.2678677e-02, -8.8416830e-02, ...,
          3.6688901e-02,  9.2964862e-03,  3.0345459e-02]], dtype=float32)}

In [18]:
class ProductIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [19]:
index = ProductIndex(data["vector"], data["name"])
index.build()

In [20]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movie to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movie to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Mask, The (1994)',
 'Clueless (1995)',
 'Sneakers (1992)',
 'Three Musketeers, The (1993)',
 'Princess Bride, The (1987)',
 'Interview with the Vampire (1994)',
 'Brady Bunch Movie, The (1995)',
 'Aladdin (1992)',
 'Conan the Barbarian (1981)']

Trees and Graphs

In [31]:
class TreesIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [34]:
index = TreesIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [35]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
similar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mask, The (1994)
* Batman (1989)
* Ready to Wear (Pret-A-Porter) (1994)
* Princess Bride, The (1987)
* Fatal Instinct (1993)
* Private Benjamin (1980)
* Interview with the Vampire (1994)
* Heavy Metal (1981)
* Hunt for Red October, The (1990)


Hierarchical Navigable Small World Algorithm(HNSW)

In [25]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [29]:
index = HNSWIndex(data["vector"], data["name"])
index.build()

In [30]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar stack to {movie_name} are:\n* {simlar_movie_questions}")

The most similar stack to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mask, The (1994)
* Batman (1989)
* Ready to Wear (Pret-A-Porter) (1994)
* Princess Bride, The (1987)
* Fatal Instinct (1993)
* Private Benjamin (1980)
* Interview with the Vampire (1994)
* Heavy Metal (1981)
* Hunt for Red October, The (1990)


Exhaustive search

In [21]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [22]:
index = ExhaustiveIndex(data["vector"], data["name"])
index.build()

In [24]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mask, The (1994)
* Princess Bride, The (1987)
* Heavy Metal (1981)
* Interview with the Vampire (1994)
* Clueless (1995)
* Hunt for Red October, The (1990)
* Sneakers (1992)
* Better Off Dead... (1985)
* Grease (1978)
